# Srock Strategy - Signal Event 0005 - ST-Removal Event-Based Strategy

## 1. Import Library

In [16]:
import dai
import random
import pandas as pd

## 2. Prepare Strategy Data

In [17]:
sd = '2025-12-31'
ed = '2026-02-13'

In [18]:
sql1 = f"""
    WITH 
    data_a1 AS (
        SELECT 
            date, 
            instrument
        FROM cn_stock_static_data
        WHERE date BETWEEN '{sd} 00:00:00.000' AND '{ed} 23:59:59.999'
        QUALIFY name NOT LIKE '%ST%'
        AND m_lag(name, 1) LIKE '%ST%'
        AND name NOT Like '%退%'
        AND m_lag(name, 1) NOT LIKE '%退%'
        ORDER BY date, instrument
    ),
    data_a2 AS (
        SELECT 
            date,
            LAG(date, 1) OVER (ORDER BY date) AS date_prev
        FROM all_trading_days
        WHERE market_code = 'CN'
    ),
    data_a3 AS (
        SELECT *
        FROM data_a1 JOIN data_a2 USING (date)
    )
    SELECT
        date_prev AS date,
        instrument,
        1 AS signal_buy,
        0 AS signal_sell,
    FROM data_a3
    ORDER BY date, instrument
"""

sql2 = f"""
    WITH 
    data_b1 AS (
        SELECT 
            date, 
            instrument
        FROM cn_stock_static_data
        WHERE date BETWEEN '{sd} 00:00:00.000' AND '{ed} 23:59:59.999'
        QUALIFY name LIKE '%ST%'
        AND m_lag(name, 1) NOT LIKE '%ST%'
        AND name NOT Like '%退%'
        AND m_lag(name, 1) NOT LIKE '%退%'
        ORDER BY date, instrument
    ),
    data_b2 AS (
        SELECT 
            date,
            LAG(date, 1) OVER (ORDER BY date) AS date_prev
        FROM all_trading_days
        WHERE market_code = 'CN'
    ),
    data_b3 AS (
        SELECT *
        FROM data_b1 JOIN data_b2 USING (date)
    )
    SELECT
        date_prev AS date,
        instrument,
        0 AS signal_buy,
        1 AS signal_sell,
    FROM data_b3
    ORDER BY date, instrument
"""

In [19]:
df1 = dai.query(sql1).df()
df2 = dai.query(sql2).df()
df = pd.concat([df1, df2]).sort_values(by = "date").reset_index(drop=True)
df

,date,instrument,signal_buy,signal_sell
0,2026-01-05,002055.SZ,0,1
1,2026-01-05,300091.SZ,0,1
2,2026-01-12,002388.SZ,1,0
3,2026-01-13,300376.SZ,1,0
4,2026-01-19,000821.SZ,0,1
5,2026-02-09,002731.SZ,0,1


## 3. Backtest Strategy

In [20]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    context.holding_days = 1
    context.is_trade_by_weight = False
    context.weight = 1 / len(df['instrument'].unique())

def BigTrader_Before_Trading(context, data):
    pass

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]

    if len(df_now) == 0:
        return

    instruments_buy  = set(df_now[df_now["signal_buy"]  == 1]["instrument"])
    instruments_sell = set(df_now[df_now["signal_sell"] == 1]["instrument"])
    instruments_hold = set(context.get_account_positions().keys())

    for instrument in instruments_buy - instruments_hold:
        if context.is_trade_by_weight:
            context.order_target_percent(instrument, context.weight)
        else:
            context.order(instrument, 100)
    
    for instrument in instruments_sell:
        context.order_target_percent(instrument, 0)

    for instrument in instruments_hold:
        if (data.current_dt - context.get_position(instrument).last_sale_date).days >= context.holding_days:
            context.order_target_percent(instrument, 0)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 100000 + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""open""",
    order_price_field_sell="""open""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-18 14:44:24] [info     ] bigtrader.v34 开始运行 ..
[2026-02-18 14:44:24] [info     ] 2026-01-05, 2026-02-09, , , instruments=6
[2026-02-18 14:44:24] [info     ] bigtrader module V2.2.0
[2026-02-18 14:44:24] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
[2026-02-18 14:44:24] [info     ] backtest done, raw_perf_ds:dai.DataSource("_ba929f7abaa7454eae0c5dfa7d0f28bd")


[2026-02-18 14:44:24] [info     ] bigtrader.v34 运行完成 [0.276s].
